# Kerchunk JSON File Recipe
The standard format for Kerchunk files is JSON for storing references to archive data chunks. This requires fsspec and xarray - although these do not need to be the latest, there is no need to not use fsspec 2023.6.0+ and xarray 2023.8.0+ as these will also work with Parquet.

In [1]:
import fsspec
import xarray as xr

Open a virtual filesystem object from fsspec, providing the kerchunk file as a 'reference' type file. Note that if the file is compressed using zstd as some kerchunk files are, the compression needs to be set to 'zstd'/'zst'

In [2]:
kfile = 'https://dap.ceda.ac.uk/neodc/esacci/land_surface_temperature/metadata/kerchunk/AQUA_MODIS/L3C/0.01/v3.00/monthly/ESACCI-LST-L3C-LST-MODISA-0.01deg_1MONTHLY_DAY-200207-201812-fv3.00-kr1.1.json'
mapper = fsspec.get_mapper('reference://',fo=kfile, backend_kwargs={'compression':None})

Then we can open a virtual xarray dataset object to plot or perform some processing.

In [3]:
ds = xr.open_zarr(mapper, consolidated=False)

In [5]:
ds

<xarray.Dataset>
Dimensions:          (channel: 2, time: 198, lat: 18000, lon: 36000,
                      length_scale: 1)
Coordinates:
  * channel          (channel) float32 11.0 12.0
  * lat              (lat) float32 -90.0 -89.99 -89.98 ... 89.97 89.98 89.99
  * lon              (lon) float32 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * time             (time) datetime64[ns] 2002-07-01 2002-08-01 ... 2018-12-01
Dimensions without coordinates: length_scale
Data variables: (12/14)
    dtime            (time, lat, lon) timedelta64[ns] dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    lcc              (time, lat, lon) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    lst              (time, lat, lon) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    lst_unc_loc_atm  (time, lat, lon) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    lst_unc_loc_sfc  (time, lat, lon) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    lst_unc_ran      (time, lat, lon) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    ...               ...
    n                (time, lat, lon) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    sataz            (time, lat, lon) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    satze            (time, lat, lon) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    solaz            (time, lat, lon) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    solze            (time, lat, lon) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    variance         (time, lat, lon) float32 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
Attributes: (12/44)
    Conventions:                CF-1.8
    cdm_data_type:              grid
    comment:                    These data were produced as part of the ESA L...
    creator_email:              djg20@le.ac.uk
    creator_name:               University of Leicester Surface Temperature G...
    creator_url:                https://climate.esa.int/en/projects/land-surf...
    ...                         ...
    time_coverage_resolution:   P1M
    time_coverage_start:        20020701T000000
    title:                      ESA LST CCI land surface temperature data at ...
    kerchunk_revision:          kr1.1
    kerchunk_creation_date:     031023T093248
    tracking_id:                8efceb61-53cb-4361-a3b4-e809046b9a19

Any plotting can then be done with this object.

In [6]:
ds['lst'].sel(lat=slice(51,59), lon=slice(-15,7)).mean(dim='time')

<xarray.DataArray 'lst' (lat: 800, lon: 2200)>
dask.array<mean_agg-aggregate, shape=(800, 2200), dtype=float32, chunksize=(800, 1000), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 51.01 51.01 51.03 51.03 ... 58.96 58.98 58.98 58.99
  * lon      (lon) float32 -14.99 -14.99 -14.98 -14.97 ... 6.975 6.985 6.995